### In this notebook Heart desease data set is used for the application of XGBoost algorithm.

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [ ]:
import xgboost as xgb
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler

In [ ]:
veri = pd.read_csv('../input/heart-desease-data/heart_desease_data.csv')
veri.head()
veri.sample(frac=1)

### Defining the inputs and outputs

In [ ]:
girdi = veri.iloc[:,0:13]
girdi
cikti = veri.iloc[:,13:14]
cikti

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(girdi, cikti.values.flatten(), test_size = 0.25, random_state = 10)
x_train.shape, x_test.shape, y_train.shape, y_test.shape

In [ ]:
scaler = StandardScaler().fit(x_train)
x_train_sc = scaler.transform(x_train)
x_test_sc = scaler.transform(x_test)

In [ ]:
D_train=xgb.DMatrix(data =x_train_sc, label = y_train)
D_test=xgb.DMatrix(data = x_test_sc)

In [ ]:
x_train_sc.shape

In [ ]:
yy = y_train[:,np.newaxis]

### Grid Search for the selection of the best hyper parameters

In [ ]:
parametre_secimi = {
    'max_depth':[3,5,7,8],
    'gamma':[0.1,0.6],
    'eval_metric':['mlogloss']
}
# num_class varken hata aldım.
model = xgb.XGBClassifier()

izgara_model = GridSearchCV(estimator = model, param_grid = parametre_secimi, cv = 3, n_jobs = -1, verbose = 2)
izgara_model.fit(x_train_sc,y_train)

In [ ]:
en_iyi_param = izgara_model.best_params_
en_iyi_model = xgb.train(params = en_iyi_param, dtrain=D_train)

en_iyi_model.save_model("modelim_class")
yukle = xgb.XGBClassifier()
yukle.load_model('modelim_class')
best_predict_proba = yukle.predict_proba(x_test_sc)
best_predicts = yukle.predict(x_test_sc)
sonuclarim = pd.DataFrame(best_predict_proba, columns = ['p1','p2'])
sonuclarim['gercek']=y_test
sonuclarim['tahmin']=best_predicts
sonuclarim

In [ ]:
from sklearn.metrics import roc_curve, roc_auc_score

### Evaluation of the model with different error metrics

In [ ]:
def sınıflandırma_metrikleri(y_true, y_prediction, n , norm):
    cm=confusion_matrix(y_true, y_prediction, normalize=norm)
    df_cm = pd.DataFrame(cm,  range(0,n),range(0,n))
    sns.heatmap(df_cm, annot=True,annot_kws={"size": 16})
    plt.suptitle('Confusion Matrix', x=0.44, y=1.0, ha='center', fontsize=25)
    plt.xlabel('Predict Values', fontsize =25)
    plt.ylabel('Test Values', fontsize =25)
    plt.show()
    print('Sınıflandırma Raporu :\n ', classification_report(y_true, y_prediction))

In [ ]:
sınıflandırma_metrikleri(y_test,best_predicts,2,None)

In [ ]:
logit_roc_auc = roc_auc_score(y_test, best_predicts)

# Görselleştirmek istersek
fpr, tpr, thresholds = roc_curve(y_test, best_predict_proba[:,0], pos_label = 0)
plt.figure()
plt.plot(fpr, tpr, label='Logistic Regression (area = %0.2f)' % logit_roc_auc)
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic')
plt.legend(loc="lower right")
# plt.savefig('Log_ROC')
plt.show();